In [21]:
import pandas as pd

In [22]:
# inspection date is daily!
inspections = pd.read_csv("../data/HCC_Inspections.csv")

# this data is hourly!
weather = pd.read_csv("../data/Hourly_Weather.csv")

# metadata on weather data
observations = pd.read_csv("../data/Weather_Observations.csv")

# need to aggregate and average the weather data
weather_merged = pd.merge(weather, observations[['ObsID', 'Date']], on='ObsID', how='left')
weather_merged = weather_merged.dropna()
for col in ['Temperature', 'Humidity', 'Dew_Point', 'Wind_Speed', 'Wind_Gust', 'Pressure', 'Precip']:
    weather_merged[col] = weather_merged[col].astype(str).str.strip()
    weather_merged[col] = pd.to_numeric(weather_merged[col])
weather_merged.head()



,WeatherID,ObsID,StationID,Temperature,Humidity,Dew_Point,Wind_Direction,Wind_Speed,Wind_Gust,Pressure,Precip,Condition,Sunrise,Sunset,Daylight_Hours,Date
0,1,1,1,54,45,33,VAR,7,0,29.54,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
1,2,2,1,52,47,32,NNW,3,0,29.56,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
2,3,3,1,51,48,32,CALM,0,0,29.57,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
3,4,4,1,48,56,33,CALM,0,0,29.57,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016
4,5,5,1,46,63,34,CALM,0,0,29.58,0.0,Fair,7:07:00,19:35:00,12:28:00,3/29/2016


In [37]:
aggregate_cols = ['Temperature', 'Humidity', 'Dew_Point', 'Wind_Speed', 'Wind_Gust', 'Pressure', 'Precip']


daily_weather = weather_merged.groupby(['Date', 'StationID'])[aggregate_cols].mean().reset_index()
daily_weather['Date'] = pd.to_datetime(daily_weather['Date'])
daily_weather['Date'] = daily_weather['Date'].dt.strftime('%m/%d/%Y')

weather_stations = pd.read_csv("../data/Weather_Stations.csv")
daily_weather = pd.merge(daily_weather, weather_stations[['StationID', 'Station_City']], on='StationID', how='left')
daily_weather.to_csv("../data/Daily_Weather.csv", index=False)

In [40]:
inspections['Date'] = pd.to_datetime(inspections['Date'])
inspections['Date'] = inspections['Date'].dt.strftime('%m/%d/%Y')

# gather hive and apiary information
hives = pd.read_csv("../data/Hive_Information.csv")
apiary_info = pd.read_csv("../data/Apiary_Information.csv")

hives = pd.merge(hives, apiary_info[['ApiaryID', 'Apiary', 'City']], on='ApiaryID', how='left')
inspections_merged = pd.merge(inspections, hives, on='HiveID', how='inner')
inspections_merged = inspections_merged.dropna()
daily_weather.to_csv("../data/derived/DetailedInspections.csv", index=False)